# Инициализация необходимых библиотек

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd
import joblib

# Загрузка данных для обучения модели

In [128]:
# Загрузка данных
df = pd.read_csv("Готовые данные.csv")

# Просмотр первых строк и структуры данных
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812 entries, 0 to 1811
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   comment                             1812 non-null   object 
 1   id                                  1812 non-null   float64
 2   rating                              1812 non-null   float64
 3   Вопрос решен                        1812 non-null   int64  
 4   Качество выполненной работы         1812 non-null   int64  
 5   Компетентность работника в общение  1812 non-null   int64  
 6   Не удовлетворен                     1812 non-null   int64  
 7   Скорость выполненной работы         1812 non-null   int64  
 8   Не понравился результат             1812 non-null   int64  
 9   Не понравилось качество             1812 non-null   int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 141.7+ KB


(                                        comment         id  rating  \
 0                                       спасибо  2945792.0     5.0   
 1                                      спасибо!  3234340.0     5.0   
 2                                      Отлично   3380332.0     5.0   
 3  Благодарю за оперативное решение проблемы !   3381812.0     5.0   
 4         Прекрасный специалист! Побольше таких  3461991.0     5.0   
 
    Вопрос решен  Качество выполненной работы  \
 0             1                            0   
 1             1                            0   
 2             1                            0   
 3             1                            0   
 4             1                            1   
 
    Компетентность работника в общение  Не удовлетворен  \
 0                                   0                0   
 1                                   0                0   
 2                                   0                0   
 3                                   0  

### Разделение данных для обучения моделей

In [129]:
X_raw = df["comment"]
y = df.iloc[:, 3:]

# Удалим пустые строки и строки без меток
mask = X_raw.notnull() & (X_raw.str.strip() != "")
X_raw = X_raw[mask]
y = y[mask]
row_mask = y.sum(axis=1) > 0
X_raw = X_raw[row_mask]
y = y[row_mask]

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(X_raw)
X = X.toarray()  # сразу в массив для совместимости с Keras и др.

# Стратифицированное разбиение по каждому классу
train_indices = set()
test_indices = set()

for col in y.columns:
    pos_indices = y[y[col] == 1].index.tolist()
    np.random.shuffle(pos_indices)
    n_train = int(0.7 * len(pos_indices))
    train_indices.update(pos_indices[:n_train])
    test_indices.update(pos_indices[n_train:])

# Приводим к спискам и убираем пересечения (если важно)
train_indices = list(train_indices)
test_indices = list(set(test_indices) - set(train_indices))

# Финальные выборки
X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y.iloc[train_indices].values, y.iloc[test_indices].values

### Logistic Regression

In [130]:
model = MultiOutputClassifier(LogisticRegression(max_iter=2000))
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

print("\n=== Logistic Regression ===")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values = []
f1_values = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred[:, i])
        f1 = f1_score(y_test[:, i], y_pred[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values.append(auc)
        f1_values.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values:
    avg_roc_auc = np.mean(roc_auc_values)
    avg_f1 = np.mean(f1_values)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc:>10.3f} {avg_f1:>10.3f}")

joblib.dump(model, "logistic_model.pkl")


=== Logistic Regression ===
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.565      0.947
Качество выполненной работы                   0.628      0.385
Компетентность работника в общение            0.581      0.267
Не удовлетворен                               0.576      0.263
Скорость выполненной работы                   0.900      0.857
Не понравился результат                       0.500      0.000
Не понравилось качество                       0.623      0.364
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.625      0.440


['logistic_model.pkl']

### Random Forest

In [131]:
rf_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=1000, random_state=42))
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\n=== Random Forest ===")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values_rf = []
f1_values_rf = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred_rf[:, i])
        f1 = f1_score(y_test[:, i], y_pred_rf[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values_rf.append(auc)
        f1_values_rf.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values_rf:
    avg_roc_auc_rf = np.mean(roc_auc_values_rf)
    avg_f1_rf = np.mean(f1_values_rf)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc_rf:>10.3f} {avg_f1_rf:>10.3f}")

joblib.dump(rf_model, "random_forest_model.pkl")


=== Random Forest ===
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.608      0.946
Качество выполненной работы                   0.642      0.364
Компетентность работника в общение            0.659      0.400
Не удовлетворен                               0.615      0.356
Скорость выполненной работы                   0.915      0.870
Не понравился результат                       0.500      0.000
Не понравилось качество                       0.623      0.364
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.652      0.471


['random_forest_model.pkl']

### Linear SVC

In [132]:
svc_model = OneVsRestClassifier(LinearSVC(max_iter=10000))
svc_model.fit(X_train, y_train)
y_pred_svc = svc_model.predict(X_test)

print("\n=== Linear SVC ===")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values_svc = []
f1_values_svc = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred_svc[:, i])
        f1 = f1_score(y_test[:, i], y_pred_svc[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values_svc.append(auc)
        f1_values_svc.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values_svc:
    avg_roc_auc_svc = np.mean(roc_auc_values_svc)
    avg_f1_svc = np.mean(f1_values_svc)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc_svc:>10.3f} {avg_f1_svc:>10.3f}")

joblib.dump(svc_model, "linear_svc_model.pkl")


=== Linear SVC ===
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.642      0.950
Качество выполненной работы                   0.632      0.316
Компетентность работника в общение            0.735      0.462
Не удовлетворен                               0.659      0.449
Скорость выполненной работы                   0.915      0.870
Не понравился результат                       0.557      0.167
Не понравилось качество                       0.735      0.400
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.696      0.516


['linear_svc_model.pkl']